In [1]:
# By Justin Johnson https://github.com/jcjohnson/pytorch-examples/blob/master/nn/dynamic_net.py

import random
import torch
from torch.autograd import Variable

"""
To showcase the power of PyTorch dynamic graphs, we will implement a very strange
model: a fully-connected ReLU network that on each forward pass randomly chooses
a number between 1 and 4 and has that many hidden layers, reusing the same
weights multiple times to compute the innermost hidden layers.
"""

class DynamicNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we construct three nn.Linear instances that we will use
    in the forward pass.
    """
    super(DynamicNet, self).__init__()
    self.input_linear = torch.nn.Linear(D_in, H)
    self.middle_linear = torch.nn.Linear(H, H)
    self.output_linear = torch.nn.Linear(H, D_out)

  def forward(self, x, verbose = False):
    """
    For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
    and reuse the middle_linear Module that many times to compute hidden layer
    representations.
    Since each forward pass builds a dynamic computation graph, we can use normal
    Python control-flow operators like loops or conditional statements when
    defining the forward pass of the model.
    Here we also see that it is perfectly safe to reuse the same Module many
    times when defining a computational graph. This is a big improvement from Lua
    Torch, where each Module could be used only once.
    """
    h_relu = self.input_linear(x).clamp(min=0)
    n_layers = random.randint(0, 3)
    if verbose:
        print("The number of layers for this run is", n_layers)
        # print(h_relu)
    for _ in range(n_layers):
        h_relu = self.middle_linear(h_relu).clamp(min=0)
        if verbose:
            pass
            # print(h_relu)
    y_pred = self.output_linear(h_relu)
    return y_pred




# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 10, 1

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = criterion(y_pred, y)
  print(t, loss.data[0])

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

(0, 60.67733383178711)
(1, 60.557655334472656)
(2, 60.50690460205078)
(3, 60.20283508300781)
(4, 60.2366943359375)
(5, 60.04125213623047)
(6, 58.39849090576172)
(7, 57.80139923095703)
(8, 59.34163284301758)
(9, 59.806427001953125)
(10, 55.32948684692383)
(11, 60.001625061035156)
(12, 60.0587043762207)
(13, 53.031707763671875)
(14, 60.024757385253906)
(15, 58.687042236328125)
(16, 59.83119583129883)
(17, 59.61639404296875)
(18, 59.323822021484375)
(19, 85.74945831298828)
(20, 58.4573974609375)
(21, 74.5245132446289)
(22, 65.59202575683594)
(23, 57.28229904174805)
(24, 57.09622573852539)
(25, 43.50196075439453)
(26, 56.76669692993164)
(27, 34.932281494140625)
(28, 56.518531799316406)
(29, 57.64817810058594)
(30, 56.02167510986328)
(31, 56.90195083618164)
(32, 25.23123550415039)
(33, 59.526519775390625)
(34, 59.283817291259766)
(35, 52.73845291137695)
(36, 58.63340759277344)
(37, 58.21649169921875)
(38, 21.990779876708984)
(39, 46.5034294128418)
(40, 21.265193939208984)
(41, 49.5994796752

(380, 4.452892780303955)
(381, 4.390689849853516)
(382, 4.4334492683410645)
(383, 4.385260581970215)
(384, 4.408660888671875)
(385, 4.380099773406982)
(386, 4.378297328948975)
(387, 4.413572311401367)
(388, 4.487453937530518)
(389, 4.383909225463867)
(390, 4.425685405731201)
(391, 4.472629547119141)
(392, 4.4620537757873535)
(393, 4.447628974914551)
(394, 4.40129280090332)
(395, 4.432995319366455)
(396, 4.389641284942627)
(397, 4.3939361572265625)
(398, 4.396951198577881)
(399, 4.395326614379883)
(400, 4.422684669494629)
(401, 4.3825602531433105)
(402, 4.422395706176758)
(403, 4.367880344390869)
(404, 4.426297187805176)
(405, 4.431412696838379)
(406, 4.400362968444824)
(407, 4.396788597106934)
(408, 4.428168296813965)
(409, 4.426473140716553)
(410, 4.420991897583008)
(411, 4.389948844909668)
(412, 4.3873748779296875)
(413, 4.381434917449951)
(414, 4.379701614379883)
(415, 4.377845764160156)
(416, 4.377194881439209)
(417, 4.371018886566162)
(418, 4.3631911277771)
(419, 4.359412670135498

In [2]:
model(x)[1:5]

Variable containing:
 1.3882
 0.3893
-1.2886
 0.1647
[torch.FloatTensor of size 4x1]

In [3]:
model(x)[1:5] # another run

Variable containing:
 1.4267
 0.4167
-1.3781
 0.1412
[torch.FloatTensor of size 4x1]

In [4]:
model(x)[1:5]

Variable containing:
 1.4267
 0.4167
-1.3781
 0.1412
[torch.FloatTensor of size 4x1]

Looks consistent! Let's now try to see what's happening inside

In [5]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 3)


Variable containing:
 1.4210
 0.3838
-1.2843
 0.1529
[torch.FloatTensor of size 4x1]

In [6]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 1.3882
 0.3893
-1.2886
 0.1647
[torch.FloatTensor of size 4x1]

In [7]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 1)


Variable containing:
 1.4267
 0.4167
-1.3781
 0.1412
[torch.FloatTensor of size 4x1]

In [8]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 1.3882
 0.3893
-1.2886
 0.1647
[torch.FloatTensor of size 4x1]

In [9]:
model(x, verbose = True)[1:5]

('The number of layers for this run is', 0)


Variable containing:
 1.3882
 0.3893
-1.2886
 0.1647
[torch.FloatTensor of size 4x1]

So what's the target?

In [10]:
y[1:5]

Variable containing:
 1.8895
 0.3787
-1.3019
 0.1532
[torch.FloatTensor of size 4x1]